In [165]:
from pandas import DataFrame, Series
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [166]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [167]:
# read in the main training csv
prevapp = pd.read_csv('home-credit-default-risk/previous_application.csv', na_values=['','XNA','XAP'])

In [168]:
prevapp.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT',
       'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT',
       'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT',
       'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED',
       'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
       'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
       'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
       'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'SELLERPLACE_AREA',
       'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT', 'NAME_YIELD_GROUP',
       'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL'], dtype=object)

Left join with the main table to get the IDs of the current applicants in the main table. 
To ensure we fill in missing values only with users in the training set

In [169]:
train_main = pd.read_csv('home-credit-default-risk/application_train.csv', na_values=['','XNA','XAP'])

In [170]:
#train_main['SK_ID_CURR'].head()
#prevapp['SK_ID_CURR'].head()

prevapp_trainIds = []
prevapp_testIds = []
 
prev_ids = prevapp['SK_ID_CURR'].unique()
train_main_ids = train_main['SK_ID_CURR'].unique()

for i in prev_ids:
    if i in train_main_ids:
        prevapp_trainIds.append(i)
    else:
        prevapp_testIds.append(i)

In [171]:
temp_df = pd.DataFrame(prevapp_trainIds, columns=["SK_ID_CURR"])
temp_df.to_csv('prevapp_train_IDs.csv', index=False)

In [172]:
temp_df = pd.DataFrame(prevapp_testIds, columns=["SK_ID_CURR"])
temp_df.to_csv('prevapp_test_IDs.csv', index=False)

In [173]:
prevapp['SK_ID_CURR'].nunique()

338857

In [174]:
len(prevapp_trainIds)

291057

In [175]:
338857-291057

47800

In [176]:
len(prevapp_testIds)

47800

Split prevapp data into train and test based on above IDs

In [177]:
prevapp_train = pd.DataFrame()

In [178]:
prevapp_trainIds_DF = pd.DataFrame(prevapp_trainIds, columns=["ID_CURR"])

In [179]:
prevapp_train = pd.merge(prevapp_trainIds_DF,prevapp, left_on='ID_CURR', right_on='SK_ID_CURR',how='left')

In [180]:
prevapp_train.shape

(1413701, 38)

In [181]:
#Checked there were no missing values in the new ID column
prevapp.shape

(1670214, 37)

In [182]:
prevapp_train.to_csv('prevapp_train.csv', index=False)

Fill in  missing values in prevapp_train

In [183]:
(prevapp_train.isnull().sum() / len(prevapp_train)*100).sort_values(ascending = False)[:38]

RATE_INTEREST_PRIMARY          99.661102
RATE_INTEREST_PRIVILEGED       99.661102
NAME_CASH_LOAN_PURPOSE         95.797343
CODE_REJECT_REASON             81.340467
NAME_PRODUCT_TYPE              63.666433
NAME_GOODS_CATEGORY            56.391627
RATE_DOWN_PAYMENT              53.019698
AMT_DOWN_PAYMENT               53.019698
NAME_SELLER_INDUSTRY           50.599243
NAME_TYPE_SUITE                49.138538
NFLAG_INSURED_ON_APPROVAL      39.690571
DAYS_LAST_DUE                  39.690571
DAYS_LAST_DUE_1ST_VERSION      39.690571
DAYS_FIRST_DUE                 39.690571
DAYS_FIRST_DRAWING             39.690571
DAYS_TERMINATION               39.690571
NAME_PAYMENT_TYPE              36.992264
NAME_YIELD_GROUP               30.359885
AMT_GOODS_PRICE                22.602021
AMT_ANNUITY                    21.731469
NAME_PORTFOLIO                 21.731116
CNT_PAYMENT                    21.731116
NAME_CLIENT_TYPE                0.109146
PRODUCT_COMBINATION             0.022140
NAME_CONTRACT_TY

In [184]:
columns = ['RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED','NAME_CASH_LOAN_PURPOSE','RATE_DOWN_PAYMENT','AMT_DOWN_PAYMENT','NAME_TYPE_SUITE','NFLAG_INSURED_ON_APPROVAL','DAYS_LAST_DUE_1ST_VERSION','NAME_PAYMENT_TYPE','AMT_GOODS_PRICE','AMT_ANNUITY','PRODUCT_COMBINATION','SK_ID_CURR','SK_ID_PREV','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT','CHANNEL_TYPE','SELLERPLACE_AREA']
prevapp_train_cols = pd.DataFrame()
prevapp_train_cols = prevapp_train.drop(columns, 1)

In [185]:
prevapp_train_cols.shape

(1413701, 19)

In [186]:
(prevapp_train_cols.isnull().sum() / len(prevapp_train_cols)*100).sort_values(ascending = False)

CODE_REJECT_REASON        81.340467
NAME_PRODUCT_TYPE         63.666433
NAME_GOODS_CATEGORY       56.391627
NAME_SELLER_INDUSTRY      50.599243
DAYS_LAST_DUE             39.690571
DAYS_TERMINATION          39.690571
DAYS_FIRST_DRAWING        39.690571
DAYS_FIRST_DUE            39.690571
NAME_YIELD_GROUP          30.359885
CNT_PAYMENT               21.731116
NAME_PORTFOLIO            21.731116
NAME_CLIENT_TYPE           0.109146
NAME_CONTRACT_TYPE         0.022140
AMT_CREDIT                 0.000071
DAYS_DECISION              0.000000
NAME_CONTRACT_STATUS       0.000000
NFLAG_LAST_APPL_IN_DAY     0.000000
AMT_APPLICATION            0.000000
ID_CURR                    0.000000
dtype: float64

Remove columns to check product type and try to fill in missing seperately

In [187]:
prevapp_train_prod_type = pd.DataFrame()

In [188]:
prevapp_train_prod_type = prevapp_train_cols[['ID_CURR','NAME_CONTRACT_TYPE','NAME_PRODUCT_TYPE','NAME_GOODS_CATEGORY','NAME_SELLER_INDUSTRY','NAME_PORTFOLIO']].copy()

In [189]:
prevapp_train_prod_type.shape

(1413701, 6)

In [190]:
(prevapp_train_prod_type.isnull().sum() / len(prevapp_train_prod_type)*100).sort_values(ascending = False)

NAME_PRODUCT_TYPE       63.666433
NAME_GOODS_CATEGORY     56.391627
NAME_SELLER_INDUSTRY    50.599243
NAME_PORTFOLIO          21.731116
NAME_CONTRACT_TYPE       0.022140
ID_CURR                  0.000000
dtype: float64

Same thing, remove columns for days_drawing and due variables for now

In [191]:
prevapp_train_days_info = pd.DataFrame()

In [192]:
prevapp_train_days_info = prevapp_train_cols[['ID_CURR','DAYS_FIRST_DRAWING','DAYS_FIRST_DUE','DAYS_LAST_DUE','DAYS_TERMINATION']].copy()

In [193]:
prevapp_train_days_info.shape

(1413701, 5)

In [194]:
(prevapp_train_days_info.isnull().sum() / len(prevapp_train_days_info)*100).sort_values(ascending = False)

DAYS_TERMINATION      39.690571
DAYS_LAST_DUE         39.690571
DAYS_FIRST_DUE        39.690571
DAYS_FIRST_DRAWING    39.690571
ID_CURR                0.000000
dtype: float64

remove these 8 columns from prevapp_train_cols for now

In [233]:
columns_2 = ['NAME_PRODUCT_TYPE','NAME_CONTRACT_TYPE','NAME_GOODS_CATEGORY','NAME_SELLER_INDUSTRY','NAME_PORTFOLIO','DAYS_FIRST_DRAWING','DAYS_FIRST_DUE','DAYS_LAST_DUE','DAYS_TERMINATION']
prevapp_train_imp_cols = pd.DataFrame()
prevapp_train_imp_cols = prevapp_train_cols.drop(columns_2, 1)

In [234]:
prevapp_train_imp_cols.shape

(1413701, 10)

In [235]:
(prevapp_train_imp_cols.isnull().sum() / len(prevapp_train_imp_cols)*100).sort_values(ascending = False)

CODE_REJECT_REASON        81.340467
NAME_YIELD_GROUP          30.359885
CNT_PAYMENT               21.731116
NAME_CLIENT_TYPE           0.109146
AMT_CREDIT                 0.000071
DAYS_DECISION              0.000000
NAME_CONTRACT_STATUS       0.000000
NFLAG_LAST_APPL_IN_DAY     0.000000
AMT_APPLICATION            0.000000
ID_CURR                    0.000000
dtype: float64

In [236]:
categorical_columns = ['CODE_REJECT_REASON','NAME_YIELD_GROUP','NAME_CLIENT_TYPE','NAME_CONTRACT_STATUS','NFLAG_LAST_APPL_IN_DAY']
for i in categorical_columns:
    prevapp_train_imp_cols[i] = prevapp_train_imp_cols[i].astype('category')

In [237]:
prevapp_train_imp_cols.dtypes

ID_CURR                      int64
AMT_APPLICATION            float64
AMT_CREDIT                 float64
NFLAG_LAST_APPL_IN_DAY    category
NAME_CONTRACT_STATUS      category
DAYS_DECISION                int64
CODE_REJECT_REASON        category
NAME_CLIENT_TYPE          category
CNT_PAYMENT                float64
NAME_YIELD_GROUP          category
dtype: object

Fill in missing values:

In [238]:
prevapp_train_imp_cols[prevapp_train_imp_cols['AMT_CREDIT'].isnull()]

,ID_CURR,AMT_APPLICATION,AMT_CREDIT,NFLAG_LAST_APPL_IN_DAY,NAME_CONTRACT_STATUS,DAYS_DECISION,CODE_REJECT_REASON,NAME_CLIENT_TYPE,CNT_PAYMENT,NAME_YIELD_GROUP
233370,438387,0.0,NaN,1,Approved,-608,NaN,Repeater,0.0,NaN


In [239]:
#AMT_CREDIT:

prevapp_train_imp_cols['AMT_CREDIT'].fillna(0, inplace = True)

In [240]:
prevapp_train_imp_cols.iloc[233370]

ID_CURR                     438387
AMT_APPLICATION                  0
AMT_CREDIT                       0
NFLAG_LAST_APPL_IN_DAY           1
NAME_CONTRACT_STATUS      Approved
DAYS_DECISION                 -608
CODE_REJECT_REASON             NaN
NAME_CLIENT_TYPE          Repeater
CNT_PAYMENT                      0
NAME_YIELD_GROUP               NaN
Name: 233370, dtype: object

In [241]:
#NAME_CLIENT_TYPE

prevapp_train_imp_cols['NAME_CLIENT_TYPE'].value_counts()

Repeater     1037682
New           259540
Refreshed     114936
Name: NAME_CLIENT_TYPE, dtype: int64

In [242]:
prevapp_train_imp_cols['NAME_CLIENT_TYPE'].fillna(prevapp_train_imp_cols['NAME_CLIENT_TYPE'].mode()[0], inplace = True)

In [243]:
#CNT_PAYMENT

prevapp_train_imp_cols['CNT_PAYMENT'].fillna(prevapp_train_imp_cols['CNT_PAYMENT'].mean(), inplace = True)

In [244]:
#NAME_YIELD_GROUP

prevapp_train_imp_cols['NAME_YIELD_GROUP'].fillna(prevapp_train_imp_cols['NAME_YIELD_GROUP'].mode(), inplace = True)

In [245]:
# Dropping because it has 30% missing
prevapp_train_imp_cols = prevapp_train_imp_cols.drop('NAME_YIELD_GROUP', 1)

In [246]:
#CODE_REJECT_REASON

prevapp_train_imp_cols[(prevapp_train_imp_cols['CODE_REJECT_REASON'].isnull()) & (prevapp_train_imp_cols['NAME_CONTRACT_STATUS']=='Refused')]

,ID_CURR,AMT_APPLICATION,AMT_CREDIT,NFLAG_LAST_APPL_IN_DAY,NAME_CONTRACT_STATUS,DAYS_DECISION,CODE_REJECT_REASON,NAME_CLIENT_TYPE,CNT_PAYMENT
168,166490,24030.000,24030.000,1,Refused,-1809,NaN,New,16.051936
198,302212,35145.000,35145.000,1,Refused,-1691,NaN,New,16.051936
252,418383,0.000,180000.000,0,Refused,-2421,NaN,Repeater,0.000000
290,393511,0.000,180000.000,0,Refused,-2576,NaN,Repeater,0.000000
566,369423,19611.000,19611.000,1,Refused,-1525,NaN,Repeater,16.051936
580,282755,48420.000,48420.000,1,Refused,-1149,NaN,Repeater,16.051936
762,141689,472500.000,500566.500,1,Refused,-199,NaN,Repeater,18.000000
771,141689,382500.000,435514.500,1,Refused,-199,NaN,Repeater,36.000000
1380,308686,0.000,180000.000,0,Refused,-2504,NaN,Repeater,0.000000
1831,265757,67000.500,67000.500,1,Refused,-8,NaN,Repeater,16.051936


In [247]:
#2% of data was rejects with no reason, no fair way found to fill these in. Dropping reject reason
prevapp_train_imp_cols = prevapp_train_imp_cols.drop('CODE_REJECT_REASON', 1)

In [248]:
(prevapp_train_imp_cols.isnull().sum() / len(prevapp_train_imp_cols)*100).sort_values(ascending = False)

CNT_PAYMENT               0.0
NAME_CLIENT_TYPE          0.0
DAYS_DECISION             0.0
NAME_CONTRACT_STATUS      0.0
NFLAG_LAST_APPL_IN_DAY    0.0
AMT_CREDIT                0.0
AMT_APPLICATION           0.0
ID_CURR                   0.0
dtype: float64

# Final Table Construction:

In [347]:
prevapp_final_train = pd.DataFrame()

In [348]:
prevapp_final_train = prevapp_train_imp_cols.groupby('ID_CURR')['AMT_APPLICATION'].sum().reset_index()

In [349]:
#Number of loan applications per ID
Number_appl = pd.DataFrame()

In [350]:
Number_appl = prevapp_train_imp_cols.groupby('ID_CURR')['AMT_APPLICATION'].count().reset_index()

In [351]:
Number_appl.rename(columns={'AMT_APPLICATION':'NUM_APPLICATIONS'},inplace=True)

In [360]:
# of loans approved
pct_approved = pd.DataFrame()
pct_approved = pd.crosstab(prevapp_train_imp_cols['ID_CURR'],prevapp_train_imp_cols['NAME_CONTRACT_STATUS'])

In [365]:
pct_approved

NAME_CONTRACT_STATUS,Approved,Canceled,Refused,Unused offer
ID_CURR,,,,
100002,1,0,0,0
100003,3,0,0,0
100004,1,0,0,0
100006,5,3,1,0
100007,6,0,0,0
100008,4,1,0,0
100009,7,0,0,0
100010,1,0,0,0
100011,3,0,1,0


In [381]:
last_per_day = pd.DataFrame()
last_per_day = pd.crosstab(prevapp_train_imp_cols['ID_CURR'],prevapp_train_imp_cols['NFLAG_LAST_APPL_IN_DAY'])

In [384]:
last_per_day

NFLAG_LAST_APPL_IN_DAY,0,1
ID_CURR,,
100002,0,1
100003,0,3
100004,0,1
100006,0,9
100007,0,6
100008,0,5
100009,0,7
100010,0,1
100011,0,4


In [375]:
# Max days since decision
days_deci = pd.DataFrame()
days_deci = prevapp_train_imp_cols.groupby('ID_CURR')['DAYS_DECISION'].max().reset_index()

In [268]:
# Total amount approved
prevapp_train_imp_cols[prevapp_train_imp_cols['NAME_CONTRACT_STATUS']=="Canceled"]

,ID_CURR,AMT_APPLICATION,AMT_CREDIT,NFLAG_LAST_APPL_IN_DAY,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_CLIENT_TYPE,CNT_PAYMENT
12,122040,0.0,0.0,1,Canceled,-301,Repeater,16.051936
27,176158,0.0,0.0,1,Canceled,-218,Repeater,16.051936
33,176158,0.0,0.0,1,Canceled,-218,Repeater,16.051936
37,202054,0.0,0.0,1,Canceled,-252,Repeater,16.051936
39,202054,0.0,0.0,1,Canceled,-34,Repeater,16.051936
50,202054,0.0,0.0,1,Canceled,-7,Repeater,16.051936
60,202054,0.0,0.0,1,Canceled,-31,Repeater,16.051936
65,199383,0.0,0.0,1,Canceled,-238,Repeater,16.051936
67,175704,0.0,0.0,1,Canceled,-14,Repeater,16.051936
71,296299,0.0,0.0,1,Canceled,-21,Repeater,16.051936


In [301]:
amt_credit = prevapp_train_imp_cols[['ID_CURR','NAME_CONTRACT_STATUS','AMT_CREDIT']]
amt_credit.head(5)

,ID_CURR,NAME_CONTRACT_STATUS,AMT_CREDIT
0,271877,Approved,17145.0
1,271877,Refused,1754721.0
2,271877,Approved,119848.5
3,108129,Approved,679671.0
4,108129,Approved,512370.0


In [316]:
amt_credit.loc[ind,'AMT_CREDIT'] = 0
# amt_credit[amt_credit['NAME_CONTRACT_STATUS']!='Approved']['AMT_CREDIT'] = 0

In [318]:
amt_credit

,ID_CURR,NAME_CONTRACT_STATUS,AMT_CREDIT
0,271877,Approved,17145.000
1,271877,Refused,0.000
2,271877,Approved,119848.500
3,108129,Approved,679671.000
4,108129,Approved,512370.000
5,108129,Approved,23688.000
6,108129,Approved,71352.000
7,108129,Approved,225000.000
8,108129,Approved,33052.500
9,122040,Approved,136444.500


In [355]:
#Total amount approved for
AMT_APPROVED = pd.DataFrame()
AMT_APPROVED = amt_credit.groupby('ID_CURR')['AMT_CREDIT'].sum().reset_index()

In [356]:
AMT_APPROVED.rename(columns={'AMT_CREDIT':'AMT_APPROVED'},inplace=True)

In [352]:
prevapp_final_train

,ID_CURR,AMT_APPLICATION
0,100002,179055.000
1,100003,1306309.500
2,100004,24282.000
3,100006,2449829.340
4,100007,903181.500
5,100008,778509.000
6,100009,537192.000
7,100010,247212.000
8,100011,810931.500
9,100012,243720.000


In [354]:
prevapp_final_train = prevapp_final_train.merge(Number_appl,how ='left',left_on='ID_CURR',right_on = 'ID_CURR')

In [357]:
prevapp_final_train = prevapp_final_train.merge(AMT_APPROVED,how ='left',left_on='ID_CURR',right_on = 'ID_CURR')

In [367]:
prevapp_final_train = prevapp_final_train.merge(pct_approved,how ='left',left_on='ID_CURR',right_on = 'ID_CURR')

In [371]:
prevapp_final_train['PCT_APPROVED'] = prevapp_final_train['Approved']/prevapp_final_train['NUM_APPLICATIONS']

In [373]:
cols = ['Approved','Canceled','Refused','Unused offer']
prevapp_final_train = prevapp_final_train.drop(cols,1)

In [377]:
prevapp_final_train = prevapp_final_train.merge(days_deci,how ='left',left_on='ID_CURR',right_on = 'ID_CURR')

In [379]:
prevapp_final_train.rename(columns={'DAYS_DECISION':'MIN_DAYS_DECISION'},inplace=True)

In [385]:
prevapp_final_train = prevapp_final_train.merge(last_per_day,how ='left',left_on='ID_CURR',right_on = 'ID_CURR')

In [388]:
prevapp_final_train.drop(1,1)

,ID_CURR,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,0
0,100002,179055.000,1,179055.000,1.000000,-606,0
1,100003,1306309.500,3,1452573.000,1.000000,-746,0
2,100004,24282.000,1,20106.000,1.000000,-815,0
3,100006,2449829.340,9,1718644.500,0.555556,-181,0
4,100007,903181.500,6,999832.500,1.000000,-374,0
5,100008,778509.000,5,813838.500,0.800000,-82,0
6,100009,537192.000,7,490963.500,1.000000,-74,0
7,100010,247212.000,1,260811.000,1.000000,-1070,0
8,100011,810931.500,4,1047361.500,0.750000,-1162,0
9,100012,243720.000,4,296478.000,0.750000,-107,0


In [392]:
prevapp_final_train['PCT_APPL_ON_SAME_DAY'] = (prevapp_final_train[0]/prevapp_final_train['NUM_APPLICATIONS'])

In [393]:
prevapp_final_train

,ID_CURR,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,0,1,FLAG_LAST_APPL_IN_DAY,PCT_APPL_ON_SAME_DAY
0,100002,179055.000,1,179055.000,1.000000,-606,0,1,1.000000,0.000000
1,100003,1306309.500,3,1452573.000,1.000000,-746,0,3,1.000000,0.000000
2,100004,24282.000,1,20106.000,1.000000,-815,0,1,1.000000,0.000000
3,100006,2449829.340,9,1718644.500,0.555556,-181,0,9,1.000000,0.000000
4,100007,903181.500,6,999832.500,1.000000,-374,0,6,1.000000,0.000000
5,100008,778509.000,5,813838.500,0.800000,-82,0,5,1.000000,0.000000
6,100009,537192.000,7,490963.500,1.000000,-74,0,7,1.000000,0.000000
7,100010,247212.000,1,260811.000,1.000000,-1070,0,1,1.000000,0.000000
8,100011,810931.500,4,1047361.500,0.750000,-1162,0,4,1.000000,0.000000
9,100012,243720.000,4,296478.000,0.750000,-107,0,4,1.000000,0.000000


In [395]:
cols = [0,1,'FLAG_LAST_APPL_IN_DAY']
prevapp_final_train = prevapp_final_train.drop(cols,1)

In [396]:
prevapp_final_train

,ID_CURR,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY
0,100002,179055.000,1,179055.000,1.000000,-606,0.000000
1,100003,1306309.500,3,1452573.000,1.000000,-746,0.000000
2,100004,24282.000,1,20106.000,1.000000,-815,0.000000
3,100006,2449829.340,9,1718644.500,0.555556,-181,0.000000
4,100007,903181.500,6,999832.500,1.000000,-374,0.000000
5,100008,778509.000,5,813838.500,0.800000,-82,0.000000
6,100009,537192.000,7,490963.500,1.000000,-74,0.000000
7,100010,247212.000,1,260811.000,1.000000,-1070,0.000000
8,100011,810931.500,4,1047361.500,0.750000,-1162,0.000000
9,100012,243720.000,4,296478.000,0.750000,-107,0.000000


In [397]:
prevapp_final_train.to_csv('prevapp_final_train.csv', index=False)